In [1]:
!pip install transformers torch
!pip install --upgrade transformers bitsandbytes accelerate safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = torch.device("cuda")

In [8]:

from transformers import pipeline

pipe = pipeline("text-generation", model="Qwen/Qwen3-1.7B")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who is lebron james?'},
   {'role': 'assistant',
    'content': '<think>\nOkay, the user is asking, "Who is LeBron James?" I need to provide a comprehensive answer. Let me start by recalling the key points about LeBron James.\n\nFirst, his full name is LeBron James, and he\'s a professional basketball player. He\'s known for being a center, right? But I should mention his position as a forward or center. He\'s played for the Cleveland Cavaliers, then the Miami Heat, and now the Los Angeles Lakers. Wait, he\'s on the Lakers now, right? But he moved from Miami to the Lakers in 2014. Oh, and he\'s also a two-time NBA champion with the Cavaliers in 2016 and 2017. \n\nHe\'s also a multiple-time All-Star, All-NBA, and All-Defender. His stats are impressive, like 30,000 points in his career. He\'s also a 3x NBA MVP, 4x NBA Finals MVP, and 4x NBA champion. That\'s a lot of accolades. \n\nHe\'s a philanthropist, has a foundation, and is known fo

In [28]:
rules = """You are Player {} in a 5 round Prisoner’s Dilemma. This is round {}. The payoff matrix is:
  Both you and opponent cooperate → each gets 3 points
  You cooperate and opponent defects → you get 0; opponent gets 5
  You defect and opponent cooperates → you get 5; opponent gets 0
  Both you and opponent defect → each gets 1 point
Based on your opponent's previous decisions, decide whether to [Cooperate] or [Defect] to ultimately maximize your points. This is a low level game, so keep your reasoning concise and under 100 words.
Please put your final answer, either [Cooperate] or [Defect] within [   ].\n"""
prevgames = "No previous rounds since it is round 1"

def doprompt(playnum, roundnum):
    temp = rules.format(playnum, roundnum) + "\n" + prevgames
    messages = [
        {"role": "user", "content": temp},
    ]
    resp = pipe(messages, max_new_tokens=1000)[0]['generated_text'][1]['content']
    print(resp)
    if "[Cooperate]" in resp:
        return "Cooperate"
    elif "[Defect]" in resp:
        return "Defect"
    else:
        return "Error"""



In [5]:
curscore1 = 0
curscore2 = 0
update = "Round {}. Player 0 chose to {}, Player 1 chose to {}. Score among all the rounds: Player 0: {}, Player 1: {}."
def prompt(roundnum):
  global curscore1
  global curscore2
  global update
  global prevgames
  a = doprompt(0, roundnum)
  b = doprompt(1, roundnum)
  if a == "Cooperate" and b == "Cooperate":
    curscore1 += 3
    curscore2 += 3
  elif a == "Cooperate" and b == "Defect":
    curscore1 += 0
    curscore2 += 5
  elif a == "Defect" and b == "Cooperate":
    curscore1 += 5
    curscore2 += 0
  else:
    curscore1 += 1
    curscore2 += 1
  if (prevgames == "No previous rounds since it is round 1"):
    prevgames = "Previous rounds: \n"
  prevgames += update.format(roundnum, a, b, curscore1, curscore2) + "\n"


In [21]:

print(a)

<think>
Okay, the user is asking "Who is LeBron James?" I need to provide a clear and concise answer. Let me start by recalling the key points about LeBron James.

First, he's a professional basketball player. He's the most famous NBA player, right? He's from Cleveland, Ohio. He was drafted in 2003. He played for the Cleveland Cavaliers, then the Miami Heat, and now the Los Angeles Lakers. 

He's known for his skills in scoring, passing, and leadership. He's been a key player in multiple NBA championships, like the 2016 and 2017 titles with the Heat. His career has been highlighted by his longevity and consistent performance. 

He's also a philanthropist, involved in various causes like education and community programs. His personal life includes being a father and a husband. 

I should mention his achievements, like being the all-time leading scorer in NBA history, his MVP awards, and his awards. Also, his impact on the game and his legacy. 

Wait, should I mention his current team? H

In [22]:
doprompt(0, 1)

'<think>\nOkay, let\'s see. It\'s round 1 of a 5-round Prisoner\'s Dilemma. The payoff matrix is given. Both cooperate: 3 points each. If I cooperate and they defect, I get 0, they get 5. If I defect and they cooperate, I get 5, they get 0. Both defect: 1 point each.\n\nSince it\'s the first round, I don\'t have any history. The question is, what should I do? Well, in the first round, the standard strategy in PD is often to cooperate, but sometimes defect. But since it\'s the first round, there\'s no prior action. But the problem says to consider the opponent\'s previous decisions. Wait, but since it\'s the first round, there are no previous decisions. So maybe the default is to cooperate? Or maybe the question is a bit tricky.\n\nWait, the problem says "based on your opponent\'s previous decisions." But since it\'s the first round, there are no previous decisions. So maybe the answer is to cooperate, as it\'s the standard first move in PD. But sometimes in some versions, people might 

In [27]:
curscore1 = 0
curscore2 = 0
prevgames = ""
for i in range(5): #num rounds
  prompt(i+1)
print(prevgames)

<think>
Okay, let's see. I need to decide whether to cooperate or defect in round 5 of the Prisoner's Dilemma. The opponent has been defecting every round so far. In each previous round, if I cooperate and they defect, I get 0. But since they've been defecting, maybe they're trying to maximize their own points. Wait, in round 1, Player 1 got 5 by defecting. Round 2, they got 10, round 3 15, round 4 20. So Player 1 has been defecting and getting more points each round. 

If I cooperate, I get 0 this round, but if I defect, I get 5. But wait, the payoff matrix says if both defect, each gets 1. But in the previous rounds, Player 1 has been defecting, so if I defect, I get 5. But if I cooperate, I get 0. But since Player 1 is defecting all the time, maybe they're not going to cooperate. So if I defect, I get 5, which is better than 0. But wait, maybe there's a better strategy. Wait, in the last round, the scores are Player 0: 0, Player 1: 20. So Player 1 is getting 20 points by defecting. 